In [18]:
import cv2
import imutils
import numpy as np

image = cv2.imread("5.jpg")

In [2]:
class PageExtractor:

    def __init__():
        pass

In [19]:
class Resizer:
    """ Resizes the Image """
    def __init__(self, max_height):
        self.max_height = max_height

    def __call__(self, image):
        
        if image.shape[0] <= self.max_height : return image

        ratio = round(self.max_height/image.shape[0] ,3)
        width = int(image.shape[1] * ratio)

        resized = cv2.resize(image, (width, self.max_height), interpolation=cv2.INTER_AREA)

        cv2.imwrite(r'output/1.jpg',resized)
        cv2.imshow("resized",resized)
        cv2.waitKey(0)
        return resized

resizer = Resizer(800)
image = resizer(image)

In [20]:
class FastDenoise:
    """This Denoise the provided image by using the fastNlMeansDenoising method"""

    def __init__(self, strength=False, save_output=True):
        self.strength = strength
        self.save_output = save_output

    def __call__(self, image):
        
        if self.strength:
            denoised = cv2.fastNlMeansDenoisingColored(image,h=self.strength)
            if self.save_output:
                cv2.imwrite(r"output/denoise.jpg",denoised)
                return denoised
        else:
            for strength in range(11):
                denoised = cv2.fastNlMeansDenoisingColored(image,h=strength)
                if self.save_output:
                    cv2.imwrite(f"output//denoise_{strength}.jpg",denoised)

fastdenoise = FastDenoise(strength=4)
image = fastdenoise(image)

In [51]:
class OtsuThresholder:
    def __init__(self, thresh1 = 0, thresh2 = 255, output_process = True):
        self.output_process = output_process
        self.thresh1 = thresh1
        self.thresh2 = thresh2


    def __call__(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        T_, thresholded = cv2.threshold(image, self.thresh1, self.thresh2, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        if self.output_process: cv2.imwrite('output/thresh.jpg', thresholded)
        return thresholded
    
thresh = OtsuThresholder()
image = thresh(image)

In [21]:
class GetEdges:
    def __init__(self, contour1=75, contour2=200, save_output= True):
        self.contour1 = contour1
        self.contour2 = contour2
        self.save_output = save_output

    def __call__(self, image):
        
        # greyimage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        greyimage = cv2.GaussianBlur(image, (5,5), 0)

        edges = cv2.Canny(greyimage, self.contour1, self.contour2)

        cv2.imwrite(r"output/edges.jpg", edges)
        cv2.imshow("original",image)
        cv2.imshow("edges", edges)
        cv2.waitKey(0)
        return edges
    
edgeDetect = GetEdges()
edges = edgeDetect(image)

In [22]:
class GetContour:
    def __init__(self, save_output =True):
        self.save_output = save_output

    def __call__(self,image ,edges):
        cnts = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

        for c in cnts:
            peri = cv2.arcLength(c,True)
            approx = cv2.approxPolyDP(c, 0.02*peri, True)

            if len(approx) == 4:
                docCnt = approx
                break

            print(docCnt)
        else:
            raise Exception
        
        # Draw the Contour
        cv2.drawContours(image.copy(), [docCnt], -1, (0, 255, 0), 2)
        cv2.imshow("Outline", image)
        if self.save_output: cv2.imwrite("output/contour.jpg", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return docCnt


getcontour = GetContour()
contour = getcontour(image,edges)

UnboundLocalError: local variable 'docCnt' referenced before assignment

In [13]:
class WarpPerspective:
    def __call__(self, image, contours):
        
        pts = contour.reshape(4,2)
        rect= np.zeros((4,2), dtype="float32")
        
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]

        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]


        (tl, tr, br, bl) =rect
        widthA = np.sqrt( (br[0]-bl[0])**2 + (br[1]-bl[1])**2 )
        widthB = np.sqrt( (tr[0]-tl[0])**2 + (tr[1]-tl[1])**2 )

        heightA = np.sqrt( (tr[0]-br[0])**2 + (tr[1]-br[1])**2 )
        heightB = np.sqrt( (tl[0]-bl[0])**2 + (tl[1]-bl[1])**2 )


        maxWidth = max(int(widthA), int(widthB))
        maxHeight = max(int(heightA), int(heightB))

        dest = np.array([
            [0,0],
            [maxWidth-1, 0],
            [maxWidth-1, maxHeight-1],
            [0, maxHeight-1]
        ], dtype='float32')

        M = cv2.getPerspectiveTransform(rect,dest)
        warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

        cv2.imwrite("output/final.jpg",warp)
        cv2.imshow("output",warp)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return warp
    
warp = WarpPerspective()
warp(image, contour)

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [102, 232, 101],
        ...,
        [ 74, 221,  78],
        [  0, 255,   0],
        [  0, 255,   0]],

       ...,

       [[  0, 255,   0],
        [ 20, 250,  20],
        [215, 203, 216],
        ...,
        [ 74,  77,  82],
        [ 25, 194,  28],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  2, 254,   2],
        [ 46, 243,  47],
        ...,
        [  0, 255,   0],
        [  1, 253,   1],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]